
## Trial to combine middle ear and bruxism tagging to identify pure moments of middle ear activation

In [2]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import mne
from tinnsleep.reports import combine_brux_MEMA
from tinnsleep.data import CreateRaw, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.events.scoring import classif_to_burst, burst_to_episode, create_list_events
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from tinnsleep.config import Config

print("Config loaded")


Config loaded


## Loading parameters and results

In [3]:
EDF_list = Config.bruxisme_files

# change to --overwrite True if you want to force to recompute the results
%run ../compute_results.py --bruxism True --mema True --overwrite False 

mema_files= pd.read_csv("data/mema_files.csv",engine='python', sep="; ")["files_with_mema"].values

results_MEMA = pd.read_pickle("data/reports_and_datas_MEMA.pk").to_dict()
results_brux = pd.read_pickle("data/reports_and_datas_bruxism.pk").to_dict()
min_burst_joining_brux = 3 
min_burst_joining_MEMA = 0

config loaded
Performs Bruxism: <True>
Performs MEMA: <True>
Will overwrite existing results: <False>
parameters set
result files exist: Reports creation skipped.


## Bruxism + MEMA processing for pure MEMA visualisation

In [5]:
from time import time
# load file from config

start = time()

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    print("Files processed : ")
    
    #Loop on all the patient files
    for filename in EDF_list:
        #opens the raw file
        file = filename.split(os.path.sep)[-1]
        print(file, end=" ")
        if not (file in results_brux.keys() and file in results_MEMA.keys()):
            print(f"(does not have both bruxism and mema)... skipping")
        else:
            window_length_brux = results_brux[file]["parameters"]["time_interval"]
            delim_brux = results_brux[file]["parameters"]["delim"]
            window_length_MEMA = results_MEMA[file]["parameters"]["time_interval"]
            delim_MEMA = results_MEMA[file]["parameters"]["delim"]
            params_combine = dict(
                                labels_brux = results_brux[file]["labels"][0], 
                                labels_artifacts_brux = results_brux[file]["parameters"]["valid_labels"], 
                                time_interval_brux = results_brux[file]["parameters"]["time_interval"], 
                                delim_ep_brux = results_brux[file]["parameters"]["delim"], 
                                labels_MEMA = results_MEMA[file]["labels"][0],
                                labels_artifacts_MEMA = results_MEMA[file]["parameters"]["valid_labels"],
                                time_interval_MEMA = results_MEMA[file]["parameters"]["time_interval"],
                                delim_ep_MEMA = results_brux[file]["parameters"]["delim"],
                                min_burst_joining_brux=min_burst_joining_brux, 
                                min_burst_joining_MEMA=min_burst_joining_MEMA
                                )
    
            brux_comb_ep, brux_pure_ep, compt_arti_brux, MEMA_comb_ep, MEMA_pure_ep, compt_arti_MEMA = combine_brux_MEMA(**params_combine)
    
            pure_brux_bursts = classif_to_burst(brux_pure_ep, time_interval=window_length_brux)
            li_ep_brux_p = burst_to_episode(pure_brux_bursts, delim=delim_brux, min_burst_joining= min_burst_joining_brux)
            pure_brux_events = create_list_events(li_ep_brux_p, window_length_brux, len(brux_pure_ep) * window_length_brux)
    
            comb_brux_bursts = classif_to_burst(brux_comb_ep, time_interval=window_length_brux)
            li_ep_brux_c = burst_to_episode(comb_brux_bursts, delim=delim_brux, min_burst_joining= min_burst_joining_brux)
            comb_brux_events = create_list_events(li_ep_brux_c, window_length_brux, len(brux_comb_ep) * window_length_brux)
            
            comb_MEMA_bursts = classif_to_burst(MEMA_comb_ep, time_interval=window_length_brux) # window_length_brux is valid ?
            
            pure_MEMA_bursts = classif_to_burst(MEMA_pure_ep, time_interval=window_length_brux) # window_length_brux is valid ?
            
            print("rapport d'activité : ")
            print("nb episodes bruxisme combinés : ")
            print(len(li_ep_brux_c))
            print("Nb épisodes bruxisme purs : ")
            print(len(li_ep_brux_p))
            print("nb episodes MEMA combinés : ")
            print(len(comb_MEMA_bursts))
            print("Nb épisodes MEMA purs : ")
            print(len(pure_MEMA_bursts))
            
            print("rapport d'activité taux horaire : ")
            print("nb episodes bruxisme combinés : ")
            print(len(li_ep_brux_c)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            print("Nb épisodes bruxisme purs : ")
            print(len(li_ep_brux_p)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            print("nb episodes MEMA combinés : ")
            print(len(comb_MEMA_bursts)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            print("Nb épisodes MEMA purs : ")
            print(len(pure_MEMA_bursts)/ ((len(MEMA_pure_ep)*window_length_brux)/3600))
            
            for i in range(len(MEMA_comb_ep)):
                if MEMA_comb_ep[i]>0:
                    MEMA_comb_ep[i]+=3
            
            for i in range(len(MEMA_pure_ep)):
                if MEMA_pure_ep[i]>0:
                    MEMA_pure_ep[i]+=2
                    
            for i in range(len(comb_brux_events)):
                if comb_brux_events[i]>0:
                    comb_brux_events[i]+=1
                    
            plt.figure()
            plt.plot(pure_brux_events, label="pure_brux")
            plt.plot(comb_brux_events, label="comb_brux")
            plt.plot(MEMA_comb_ep, label="comb_MEMA")
            plt.plot(MEMA_pure_ep, label="pure_MEMA")
            plt.legend()
            plt.title("Where the events happen for " + file)
                        
                       

Files processed : 
1BA07_nuit_hab.edf (does not have both bruxism and mema)... skipping
1BA07_nuit_son.edf (does not have both bruxism and mema)... skipping
1CC05_nuit_hab.edf (does not have both bruxism and mema)... skipping
1CC05_nuit_son.edf (does not have both bruxism and mema)... skipping
1DA15_nuit_hab.edf (does not have both bruxism and mema)... skipping
1DA15_nuit_son.edf (does not have both bruxism and mema)... skipping
1DL12_nuit_hab.edf (does not have both bruxism and mema)... skipping
1DL12_nuit_son.edf (does not have both bruxism and mema)... skipping
1GB18_nuit_son.edf (does not have both bruxism and mema)... skipping
1GB19_nuit_hab.edf (does not have both bruxism and mema)... skipping
1GF14_nuit_hab.edf (does not have both bruxism and mema)... skipping
1GF14_nuit_son.edf (does not have both bruxism and mema)... skipping
1MF19_nuit_hab.edf (does not have both bruxism and mema)... skipping
1MF19_nuit_son.edf (does not have both bruxism and mema)... skipping
1MN09_nuit_hab.

In [6]:
         
#-----------------Pure MEMA episodes visualisation and comparison with ATM activity -----------------------------------
#Preparing raw for visualisation
picks_chan = ['Airflow', '1', '2']           # subset of EMG electrodes
raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"], picks_chan, ch_types=['misc','emg','emg'])        # pick channels and load
scalings = dict(misc=5, emg=5e-6)

raw  = raw.filter(20., 99., n_jobs=4, 
                  fir_design='firwin', filter_length='auto', phase='zero-double',
                  picks=['1', '2'])
#Annotating the raw
raw = CleanAnnotations(raw)
dict_annotations = {1: "b_pure", 2:"b_comb", 3:"M_pure", 4:"M_comb"}
raw = AnnotateRaw_sliding(raw, MEMA_comb_ep , 
                dict_annotations=dict_annotations, duration=duration_brux, interval=interval_brux)

raw = AnnotateRaw_sliding(raw, comb_brux_events , 
                dict_annotations=dict_annotations, duration=duration_brux, interval=interval_brux)
raw = AnnotateRaw_sliding(raw, pure_brux_events , 
                dict_annotations=dict_annotations, duration=duration_brux, interval=interval_brux)
raw = AnnotateRaw_sliding(raw, MEMA_pure_ep , 
                dict_annotations=dict_annotations, duration=duration_brux, interval=interval_brux)

print("Raw annotated")
raw.plot(scalings = scalings)
plt.title(filename)
                    

<ipython-input-6-ef8c19c94daf>:4: RuntimeWarning: 7 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Intensit? lumine', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading


NameError: name 'MEMA_comb_ep' is not defined